In [ ]:
# the same housekeeping
import os

try:
    workdir
    raise Exception("This cell should be run only once (workdir = %s)" % workdir)
except NameError:
    pass
   
basedir=os.getcwd()
workdir=basedir + '/work'

In [16]:
import mdtraj as md
import os

os.chdir(workdir)

tr = md.load('full-landmarks.pdb')
heavy_idx = tr[0].top.select("element != H")

heavy = tr.atom_slice(heavy_idx)
heavy.center_coordinates()
first = heavy.slice(0)


heavy.save_xtc('heavy-landmarks.xtc')
first.save_pdb('heavy1.pdb')




In [ ]:
import anncolvar

epochs = 500

# anncolvar defaults
nlayers = 1
layers = [32, 0, 0]
actfun = ['sigmoid','linear','linear']
optim = 'adam'
loss = 'mean_squared_error'
batch = 256



os.chdir(workdir)

rg = md.compute_rg(first)

# magic -- seems to be safe
box = rg[0] * 5

!gmx editconf -f heavy1.pdb -o reference.pdb -box {box} {box} {box} -c 

# XXX: too much stdout 
for col in [1,2]:
    anncolvar.anncollectivevariable('heavy-landmarks.xtc','reference.pdb','colvar.txt',col,
                                    box,box,box,.1,0,0,
                                    nlayers,*layers,
                                    *actfun,
                                    optim,loss,epochs,batch,
                                    '','','plumed%d.dat' % col)
                                    
                                    

In [31]:
import os
import re

os.chdir(workdir)
with open('plumed.dat','w') as fout:
    for col in [1,2]:
        with open('plumed%d.dat' % col) as fin:
            for line in fin:
                w = line.split()
                if w[1] == 'POSITION':
                    a = w[2].split('=')
                    line = w[0] + (' POSITION ATOM=%d ' % (heavy_idx[int(a[1])-1]+1)) + ' '.join(w[3:]) + '\n'
                if re.match('[pl][0-9_rxyz]',line):
                    line = re.sub('[pl][0-9_rxyz]','cv%d_\g<0>' % col,line)
                    fout.write(line)
                elif col == 1 and not re.match('PRINT',line):
                    fout.write(line)
  
    # XXX: hardcoded
    fout.write('PRINT ARG=cv1_l2r,cv2_l2r STRIDE=100 FILE=COLVAR\n')
    fout.write('METAD ARG=cv1_l2r,cv2_l2r SIGMA=0.1,0.1 HEIGHT=1.0 PACE=1000 BIASFACTOR=15 TEMP=300 LABEL=restraint\n')
                